In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import cv2
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error


In [7]:
# Paths to videos and directories
train_video_path = '/kaggle/input/q3-computer-vision/train.mp4'
test_video_path = '/kaggle/input/q3-computer-vision/test.mp4'
train_frames_dir = '/kaggle/input/optical-flow-dataset/train_frames'
test_frames_dir = '/kaggle/input/optical-flow-dataset/test_frames'
train_flows_dir = '/kaggle/input/optical-flow-dataset/train_flows'
test_flows_dir = '/kaggle/input/optical-flow-dataset/test_flows'
train_labels_file = '/kaggle/input/q3-computer-vision/train.txt'
test_labels_file = '/kaggle/input/q3-computer-vision/test.txt'


In [26]:
import torch
from torch.utils.data import Dataset
import os
import numpy as np
from torchvision import transforms
from PIL import Image

class CarSpeedDataset(Dataset):
    def __init__(self, frames_dir, flows_dir, labels_file, frame_ratio=0.1, transform=None):
        self.frames_dir = frames_dir
        self.flows_dir = flows_dir
        self.labels = self.load_labels(labels_file)
        self.frame_ratio = frame_ratio
        self.transform = transform
        self.use_frame = np.random.rand(len(self.labels)) < self.frame_ratio

    def load_labels(self, labels_file):
        with open(labels_file, 'r') as file:
            labels = [float(line.strip()) for line in file]
        return labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        frame_path = os.path.join(self.frames_dir, f'frame_{idx+1:04d}.jpg')
        flow_path = os.path.join(self.flows_dir, f'flow_{idx+1:04d}.jpg')

        if not os.path.exists(frame_path) or not os.path.exists(flow_path):
            print(f"Skipping index {idx}: File not found")
            return self.__getitem__((idx + 1) % len(self))

        frame = Image.open(frame_path).convert('RGB')
        flow = Image.open(flow_path).convert('RGB')

        if self.transform:
            frame = self.transform(frame)
            flow = self.transform(flow)

        # Use 10% frames and 90% flows
        if self.use_frame[idx]:
            input_data = frame
        else:
            input_data = flow

        label = self.labels[idx]

        return input_data, label

# Data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


train_dataset = CarSpeedDataset(train_frames_dir, train_flows_dir, train_labels_file, transform=data_transforms)
test_dataset = CarSpeedDataset(test_frames_dir, test_flows_dir, test_labels_file, transform=data_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)


In [27]:
import torch.nn as nn
import torch.nn.functional as F

class SpeedPredictionModel(nn.Module):
    def __init__(self):
        super(SpeedPredictionModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)  # Adjusted input size
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SpeedPredictionModel().cuda()


In [28]:
import torch.optim as optim
from tqdm import tqdm

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
        inputs, labels = inputs.cuda(), labels.float().cuda()  # Ensure labels are cast to float
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.squeeze()  # Remove unnecessary dimension
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Training Epoch 1/10:  30%|███       | 193/638 [02:01<04:36,  1.61it/s]

Skipping index 20398: File not found
Skipping index 20399: File not found


Training Epoch 1/10:  75%|███████▌  | 481/638 [05:18<01:43,  1.51it/s]

Skipping index 20399: File not found


Training Epoch 1/10: 100%|██████████| 638/638 [07:07<00:00,  1.49it/s]


...

Skipping index 20399: File not found


Training Epoch 10/10:  41%|████      | 260/638 [01:40<02:27,  2.57it/s]

Skipping index 20398: File not found
Skipping index 20399: File not found


Training Epoch 10/10: 100%|██████████| 638/638 [04:05<00:00,  2.59it/s]

Epoch 10, Loss: 0.9207510627736119


In [29]:
from sklearn.metrics import mean_absolute_error

def evaluate_model(model, test_loader):
    model.eval()
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.cuda()
            labels = labels.cuda()
            outputs = model(inputs)
            predictions.extend(outputs.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())

    mae = mean_absolute_error(ground_truths, predictions)
    print(f'Mean Absolute Error: {mae}')

evaluate_model(model, test_loader)


Skipping index 10796: File not found
Skipping index 10797: File not found
Skipping index 10797: File not found
Mean Absolute Error: 4.735054195054909


In [37]:
import cv2
import numpy as np
from tqdm import tqdm

def draw_text(frame, text, position=(10, 30), font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.5, font_color=(0, 255, 0), thickness=1):
    cv2.putText(frame, text, position, font, font_scale, font_color, thickness, cv2.LINE_AA)

def create_video_with_predictions(frames_dir, flows_dir, test_loader, model, output_path_frames, output_path_overlay, frame_size=(224, 224), fps=10):
    model.eval()
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out_frames = cv2.VideoWriter(output_path_frames, fourcc, fps, frame_size)
    out_overlay = cv2.VideoWriter(output_path_overlay, fourcc, fps, frame_size)
    
    with torch.no_grad():
        for idx, (inputs, labels) in tqdm(enumerate(test_loader), total=len(test_loader)):
            inputs = inputs.cuda()
            outputs = model(inputs)
            predictions = outputs.cpu().numpy()
            
            batch_size = inputs.size(0)
            for i in range(batch_size):
                actual_idx = idx * test_loader.batch_size + i
                frame_path = os.path.join(frames_dir, f'frame_{actual_idx+1:04d}.jpg')
                flow_path = os.path.join(flows_dir, f'flow_{actual_idx+1:04d}.jpg')
                
                frame = cv2.imread(frame_path)
                flow = cv2.imread(flow_path)
                
                if frame is None or flow is None:
                    continue
                
                frame = cv2.resize(frame, frame_size)
                flow = cv2.resize(flow, frame_size)
                
                predicted_speed = predictions[i][0]
                
                draw_text(frame, f'Predicted Speed: {predicted_speed:.2f}', position=(10, 20))
                overlay = cv2.addWeighted(frame, 0.7, flow, 0.3, 0)
                draw_text(overlay, f'Predicted Speed: {predicted_speed:.2f}', position=(10, 20))
                
                out_frames.write(frame)
                out_overlay.write(overlay)
    
    out_frames.release()
    out_overlay.release()

# Directories and model loading (adjust these paths as needed)
test_frames_dir = '/kaggle/input/optical-flow-dataset/test_frames'
test_flows_dir = '/kaggle/input/optical-flow-dataset/test_flows'
output_path_frames = 'predicted_speed_frames.mp4'
output_path_overlay = 'predicted_speed_overlay.mp4'

# Assuming model and test_loader are already defined and loaded
create_video_with_predictions(test_frames_dir, test_flows_dir, test_loader, model, output_path_frames, output_path_overlay)


100%|██████████| 338/338 [03:31<00:00,  1.60it/s][ WARN:0@6321.214] global loadsave.cpp:241 findDecoder imread_('/kaggle/input/optical-flow-dataset/test_frames/frame_10798.jpg'): can't open/read file: check file path/integrity
[ WARN:0@6321.214] global loadsave.cpp:241 findDecoder imread_('/kaggle/input/optical-flow-dataset/test_flows/flow_10798.jpg'): can't open/read file: check file path/integrity


Skipping index 10796: File not found
Skipping index 10797: File not found
Skipping index 10797: File not found


In [39]:
# output Files Link :